<a href="https://colab.research.google.com/github/ChetanAIML/Capstone/blob/main/Human_Face_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

# Step 1: Exploratory Data Analysis (EDA)

# Load the CSV file containing facial bounding box coordinates
df = pd.read_csv('facial_bounding_boxes.csv')

# Check for missing values
print(df.isnull().sum())

# Descriptive statistics of bounding box coordinates
print(df.describe())

# Visualize the distribution of bounding box coordinates
plt.hist(df['x0'], bins=20, edgecolor='black')
plt.title('Distribution of x0 coordinates')
plt.show()

# Visualize images with bounding boxes
for i in range(10):
    image_path = df.loc[i, 'image_path']
    image = plt.imread(image_path)
    x0, y0, x1, y1 = df.loc[i, ['x0', 'y0', 'x1', 'y1']]
    plt.imshow(image)
    plt.plot([x0, x1], [y0, y0], 'b-', linewidth=2)
    plt.plot([x0, x0], [y0, y1], 'b-', linewidth=2)
    plt.plot([x1, x1], [y0, y1], 'b-', linewidth=2)
    plt.plot([x0, x1], [y1, y1], 'b-', linewidth=2)
    plt.show()

# Step 2: Split Dataset into Train, Test, and Validation Sets

# Divide the data into training, testing, and validation sets
train_data = df.sample(frac=0.7)
test_data = df.drop(train_data.index)
val_data = test_data.sample(frac=0.25)
test_data = test_data.drop(val_data.index)

# Step 3: Train, Evaluate, and Plot Accuracy and Loss

# Define the CNN architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPool2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPool2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='linear')
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
train_generator = train_datagen.flow_from_dataframe(train_data, target_size=(224, 224), batch_size=32, class_mode='rectangles')

# Train the model
history = model.fit(train_generator, epochs=10, validation_data=val_data)

# Plot accuracy and loss curves
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Step 4: Face Detection and Bounding Box Drawing

# Load the image to detect faces in
image = plt.imread('test_image.jpg')

# Preprocess the image
image = tf.image.resize(image, (224, 224))
image = image.astype('float32') / 255.0

# Make predictions
predictions = model.predict(np.expand_dims(image, axis=0))[0]

# Draw bounding boxes on the image
for prediction in predictions:
    x0, y0, x1, y1 = prediction
    x0, y0, x1, y1 = int(x0), int(y0), int(x1), int(y1)
    plt.plot([x0, x1], [y0, y0], 'b-', linewidth=2)
    plt.plot([x0, x0], [y0, y1], 'b-', linewidth=2)
    plt.plot([x1, x1], [y0, y1], 'b-', linewidth=2)
    plt.plot([x0, x1], [y1, y1], 'b-', linewidth=2)
    plt.imshow(image)
    plt.show()
